# Generate Initial Configuration

This script will generate the initial configuration file for your polymer simulations. 

In [1]:
import numpy as np 
import gsd.hoomd
import hoomd

First let's define our input variables, `chl` which is the chain length or number of beads per polymer and `N` which is the number of *total atoms* in the system. Once we set those we can determine the number of polymer chains `nch = N/chl`. We can also create an array to store particle positions in and set the box size.

In [2]:
chl = 2
N = 480 
nch = int(N/chl)

#allocate arrays
pos = np.zeros((N,3), 'd')
L = np.array([15, 15, 15])

Now we can determine the high and low bounds of the simulation box, as well as the position of the first particle, which will be in the corner of the simulation box.

In [ ]:
lo = -L/2.0 #lower box bounds
hi = L/2.0 #upper box bounds

xo, yo, zo = -L[0]/2.0, -L[1]/2.0, -L[2]/2.0 #initial x, y, and z positions

Next, we can place the particles on a lattice by snaking through the box, checking that a new particle will fit, and then placing it before moving on to the next spot on the lattice. 

In [ ]:
dx = 1.0 #spacing between particles in the x direction
dy = 1.0 #spacing between particles in the y direction

# Put particles on a lattice 
for i in range(0, N):
    pos[i,0], pos[i,1], pos[i,2] = xo, yo, zo
    xo += dx
    if (xo*dx >= hi[0]):
        yo += dy
        xo -=dx
        dx *= -1.0

        if (yo*dy >= hi[1]):
            yo -= dy
            zo += 1.0
            dy *= -1.0

Finally, we can store all this information in a GSD frame and write it to our initial configuration file. 

In [ ]:

frame = gsd.hoomd.Frame()
frame.configuration.box = [L[0], L[1], L[2], 0, 0, 0]

frame.particles.N = N
frame.particles.types=['typeA']
frame.particles.typeid=[0]*N
frame.particles.position=pos

#here we assign the bonds between particles on the same polymer chain
frame.bonds.N = (nch*(chl-1))
bond = 0
bond_groups = []
for i in range(0, nch):
    for j in range(0, chl-1):
        i1 = chl*i + j
        i2 = i1+1
        bond_groups.append([i1, i2])
        bond += 1
frame.bonds.group = np.array(bond_groups)
frame.bonds.types = ['A-A']
frame.bonds.typeid = [0]*len(bond_groups)

#note: the filename is an f string, meaning it will write the value of the variable 
# within the brackets, so you don't need to change it manually
with gsd.hoomd.open(name=f'chl{chl}_init.gsd', mode='w') as f: 
    f.append(frame)
f.close()